<center>

EcoCommons GitHub Notebooks ![EcoCommons](https://www.ecocommons.org.au)
    
</center>


# EcoCommons Notebook 
## Species Distribution Model
### Genralised Linear Model GLM 


---
Author details:     
Contact details: comms@ecocommons.org.au  
Copyright statement: This script is the product of the EcoCommons platform.   
                     Please refer to the EcoCommons website for more details:   
                     <https://www.ecocommons.org.au/>  
Date: August 2024  

---

#### Script and data info: 

This notebook provides an example of how to run a simple version SDM 

In the near future, this material may form part of comprehensive support materials available to EcoCommons users.
 
If you have any corrections or suggestions to improve the effeciengy, please [contact the EcoCommons](mailto:comms@ecocommons.org.au) support and communications team.

In [106]:
#packages for the analysis
# SLM : do we need to install these packages? Please consider using renv to manage the dependencies
library(dismo)
library(dplyr) # I prefer reading tidyverse (ggplot already included)
library(ggplot2)
library(raster)

In [ ]:
setwd("/Users/qcifecocommons/Documents/RDIR/wd4liteSDM/sdm_data")
getwd()
#SLM: I cannot access this repository. Please consider using a public repository or providing the data in a different way

In [ ]:
#read occurance data
tree_kangaroo_data <-read.csv("Tree_kangaroo.csv")
#check data class
sapply(tree_kangaroo_data, class)
#change data class to numeric
tree_kangaroo_data <- tree_kangaroo_data %>% mutate(across(where(is.character), as.numeric))
#check again
sapply(tree_kangaroo_data, class)  #SLM:is there a way to automate this check?
#assign CRS
# SLM: what happened here? I cannot see the CRS being assigned


In [ ]:
# SLM: this needs a bit of explanation. What are we doing here :) ?
file.exists("env_forecast.gri")
env_data_current <-stack("env_current.grd")
env_data_forecast <-stack("env_forecast.gri")

In [ ]:
#SLM: this can be changed into a function instead of 4 individual lines
plot(env_data_current$tmin)
plot(env_data_current$precip)
plot(env_data_forecast$tmin)
plot(env_data_forecast$precip)

In [111]:
tk_locations <- select(tree_kangaroo_data,lon,lat)
tk_env <-extract(env_data_current,tk_locations)
tree_kangaroo_data <- cbind(tree_kangaroo_data, tk_env)



In [ ]:
# Check CRS

crs(env_data_current)
crs(tk_locations)

In [ ]:
ggplot(tree_kangaroo_data, 
       mapping = aes(x =tmin,y = precip, color = present)) +
  geom_point()

In [ ]:
#GLM
#multivariate ligistic regression
#Occurance on X & envir on Y
#Probability low or high for multiple variables

logistic_regr_model <- glm(present ~ tmin + precip,
                           family = binomial(link = "logit"),
                           data = tree_kangaroo_data)
summary(logistic_regr_model)

# Model evaluation

In [ ]:
presence_data <-filter(tree_kangaroo_data,present ==1)
absence_data <-filter(tree_kangaroo_data,present ==0)

evaluation = evaluate(presence_data,absence_data,logistic_regr_model)
plot(evaluation,"ROC")

In [ ]:
#absence from large scales can be problematic


predictions = predict(env_data_current,
                      logistic_regr_model,
                      type = "response") 
plot(predictions, ext = extent(138, 154, -30, -10))

points(presence_data[c("lon","lat")],pch ="+", cex = 0.5)



In [ ]:
#40% threshold
plot(predictions>0.3, ext =extent(138, 154, -30, -10))
# Calculate the threshold
tr <- threshold(evaluation, stat = "prevalence")
# Plot the predictions with the specified threshold and extent
plot(predictions > tr, ext = extent(138, 154, -30, -10))
points(presence_data[c("lon", "lat")], pch = "+", cex = 0.5)


In [ ]:
str(env_data_forecast)
summary(env_data_forecast)
summary(env_data_current)
model_vars <- names(coef(logistic_regr_model))
missing_vars <- setdiff(model_vars, names(env_data_forecast))

if (length(missing_vars) > 0) {
  print(paste("Missing variables in env_data_forecast:", paste(missing_vars, collapse = ", ")))
}


In [ ]:
#future

forecasts = predict(env_data_forecast,
                    logistic_regr_model
                    ,type = "response")
extent_values <- extent(138, 154, -30, -10)
plot(predictions, ext = extent_values)
plot(forecasts > tr, ext = extent(138, 154, -30, -10))
plot(forecasts - predictions, ext = extent(138, 154, -30, -10))

